In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from typing import List
from datasetsforecast.long_horizon import LongHorizon
import pandas as pd
import numpy as np
import torch
from typing import Optional

import os
import pathlib

from torch.utils.data import Dataset, DataLoader, Subset
from tqdm.auto import tqdm

from dataset import LongHorizonUnivariateDataModule, LongHorizonUnivariateDataset
from dataset import ElectricityUnivariateDataModule, ElectricityUnivariateDataset
import pytorch_lightning as pl


import bisect

/opt/conda/lib/python3.10/site-packages/lightning_fabric/__init__.py:29: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)


In [3]:
mhlv = pd.read_csv('data/electricity/datasets/mhlv/raw/MHLV.csv')
mhlv = mhlv.set_index('Datetime')
mhlv

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var26,Var27,Var28,Var29,Var30,Var31,Var32,Var33,Var34,Var35
Datetime,,,,,,,,,,,,,,,,,,,,,
01-Jan-2006 00:00:00,NaN,6297.0,1425.0,9829.0,4293.00,NaN,6631.0,52552.747253,NaN,NaN,...,10215.0,NaN,14268.000000,5318.556701,5957.0,NaN,NaN,1236.842105,3332.00,NaN
01-Jan-2006 01:00:00,NaN,6005.0,1358.0,9447.0,4186.00,NaN,6346.0,52552.747253,NaN,NaN,...,9979.0,NaN,13602.000000,5139.175258,5837.0,NaN,NaN,1209.473684,3165.00,NaN
01-Jan-2006 02:00:00,NaN,5743.0,1283.0,9062.0,4026.00,NaN,6365.0,47740.659341,NaN,NaN,...,9460.0,NaN,13027.000000,4862.886598,5592.0,NaN,NaN,1155.789474,3040.00,NaN
01-Jan-2006 03:00:00,NaN,5397.0,1175.0,8589.0,3877.00,NaN,6134.0,45529.670330,NaN,NaN,...,8833.0,NaN,12606.000000,4490.721649,5383.0,NaN,NaN,1102.105263,3021.00,NaN
01-Jan-2006 04:00:00,NaN,5213.0,1134.0,8247.0,3724.00,NaN,6361.0,44514.285714,NaN,NaN,...,8525.0,NaN,12393.000000,4212.371134,5201.0,NaN,NaN,1071.578947,2959.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31-Dec-2018 19:00:00,1260.64,7510.0,1655.0,9051.0,4191.43,6612.0,6369.0,51404.153061,4011.40,1151.58,...,13117.0,16899.22,16233.673469,6011.000000,6502.0,5761.0,16766.44,1435.000000,3093.11,30574.80
31-Dec-2018 20:00:00,1116.70,7134.0,1537.0,8825.0,4076.78,6396.0,6215.0,49750.295918,3900.60,1151.58,...,12369.0,16559.33,15440.816327,5505.000000,6220.0,5379.0,16110.11,1345.000000,2990.83,29479.67
31-Dec-2018 21:00:00,1033.34,7250.0,1471.0,9109.0,3999.01,6567.0,6093.0,49220.000000,3720.40,1158.17,...,11937.0,16087.57,15038.775510,5275.000000,6023.0,5209.0,15577.03,1286.000000,2936.71,28700.14


In [4]:
mhlv_test = pd.read_csv('data/electricity/datasets/mhlv/raw/MHL_test2.csv', 
                        header=None, names=mhlv.columns)
mhlv_test.set_index(mhlv.index[-len(mhlv_test):], inplace=True)
mhlv_test

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var26,Var27,Var28,Var29,Var30,Var31,Var32,Var33,Var34,Var35
Datetime,,,,,,,,,,,,,,,,,,,,,
01-Jan-2018 00:00:00,746.45,6414,1249,8379.0,3424.49,7311,5530,44142.346939,3146.50,799.6,...,11255,16763.59,13654.081633,4500,5332,5142.0,15714.00,1103,2734.27,24635.32
01-Jan-2018 01:00:00,626.28,6153,1189,8010.0,3281.06,7484,5496,42825.428571,3024.70,783.2,...,10956,16509.32,13045.918367,4254,5162,4908.0,15745.00,1058,2658.82,23872.12
01-Jan-2018 02:00:00,548.54,5907,1142,7713.0,3172.75,7459,5337,42276.469388,2931.40,776.8,...,10647,16109.38,12706.122449,4047,5071,4564.0,15508.00,1022,2582.76,23194.89
01-Jan-2018 03:00:00,519.03,5918,1100,7568.0,3122.35,7260,5293,42205.551020,2878.90,779.2,...,10431,16152.19,12546.938776,3905,5013,4297.0,15324.00,1010,2567.72,23071.96
01-Jan-2018 04:00:00,524.56,6033,1090,7579.0,3111.68,7285,5284,41956.897959,2896.90,780.5,...,10431,16205.08,12437.755102,3815,4987,4134.0,15516.00,1023,2562.35,23267.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31-Dec-2018 19:00:00,1260.64,7510,1655,9051.0,4191.43,6612,6369,51404.153061,4011.40,NaN,...,13117,16899.22,16233.673469,6011,6502,5761.0,16766.44,1435,3093.11,30574.80
31-Dec-2018 20:00:00,1116.70,7134,1537,8825.0,4076.78,6396,6215,49750.295918,3900.60,NaN,...,12369,16559.33,15440.816327,5505,6220,5379.0,16110.11,1345,2990.83,29479.67
31-Dec-2018 21:00:00,1033.34,7250,1471,9109.0,3999.01,6567,6093,49220.000000,3720.40,NaN,...,11937,16087.57,15038.775510,5275,6023,5209.0,15577.03,1286,2936.71,28700.14


In [5]:
np.isclose(mhlv_test, mhlv[-len(mhlv_test):]).sum(axis=0)

array([8760, 8760, 8760, 8760, 8760, 8760, 8760, 8760, 8760, 8013, 8760,
       8760, 8760, 8760, 8760, 8760, 8760, 8760, 8760, 8016, 8760, 8760,
       7294, 8760, 8760, 8760, 8760, 8760, 8760, 8760, 8760, 8760, 8760,
       8760, 8760])

In [6]:
mhlv[-len(mhlv_test):] = (mhlv[-len(mhlv_test):] + mhlv_test) / 2
mhlv[-len(mhlv_test):]

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var26,Var27,Var28,Var29,Var30,Var31,Var32,Var33,Var34,Var35
Datetime,,,,,,,,,,,,,,,,,,,,,
01-Jan-2018 00:00:00,746.45,6414.0,1249.0,8379.0,3424.49,7311.0,5530.0,44142.346939,3146.50,799.6,...,11255.0,16763.59,13654.081633,4500.0,5332.0,5142.0,15714.00,1103.0,2734.27,24635.32
01-Jan-2018 01:00:00,626.28,6153.0,1189.0,8010.0,3281.06,7484.0,5496.0,42825.428571,3024.70,783.2,...,10956.0,16509.32,13045.918367,4254.0,5162.0,4908.0,15745.00,1058.0,2658.82,23872.12
01-Jan-2018 02:00:00,548.54,5907.0,1142.0,7713.0,3172.75,7459.0,5337.0,42276.469388,2931.40,776.8,...,10647.0,16109.38,12706.122449,4047.0,5071.0,4564.0,15508.00,1022.0,2582.76,23194.89
01-Jan-2018 03:00:00,519.03,5918.0,1100.0,7568.0,3122.35,7260.0,5293.0,42205.551020,2878.90,779.2,...,10431.0,16152.19,12546.938776,3905.0,5013.0,4297.0,15324.00,1010.0,2567.72,23071.96
01-Jan-2018 04:00:00,524.56,6033.0,1090.0,7579.0,3111.68,7285.0,5284.0,41956.897959,2896.90,780.5,...,10431.0,16205.08,12437.755102,3815.0,4987.0,4134.0,15516.00,1023.0,2562.35,23267.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31-Dec-2018 19:00:00,1260.64,7510.0,1655.0,9051.0,4191.43,6612.0,6369.0,51404.153061,4011.40,NaN,...,13117.0,16899.22,16233.673469,6011.0,6502.0,5761.0,16766.44,1435.0,3093.11,30574.80
31-Dec-2018 20:00:00,1116.70,7134.0,1537.0,8825.0,4076.78,6396.0,6215.0,49750.295918,3900.60,NaN,...,12369.0,16559.33,15440.816327,5505.0,6220.0,5379.0,16110.11,1345.0,2990.83,29479.67
31-Dec-2018 21:00:00,1033.34,7250.0,1471.0,9109.0,3999.01,6567.0,6093.0,49220.000000,3720.40,NaN,...,11937.0,16087.57,15038.775510,5275.0,6023.0,5209.0,15577.03,1286.0,2936.71,28700.14


In [7]:
np.isclose(mhlv_test, mhlv[-len(mhlv_test):]).sum(axis=0)

array([8760, 8760, 8760, 8760, 8760, 8760, 8760, 8760, 8760, 8013, 8760,
       8760, 8760, 8760, 8760, 8760, 8760, 8760, 8760, 8016, 8760, 8760,
       7294, 8760, 8760, 8760, 8760, 8760, 8760, 8760, 8760, 8760, 8760,
       8760, 8760])

In [8]:
mhlv = mhlv.stack(dropna=False).reset_index()
mhlv.rename({'Datetime': 'ds', 'level_1': 'unique_id', 0: 'y'}, inplace=True, axis=1)
mhlv

/tmp/ipykernel_281/896426315.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  mhlv = mhlv.stack(dropna=False).reset_index()


,ds,unique_id,y
0,01-Jan-2006 00:00:00,Var1,NaN
1,01-Jan-2006 00:00:00,Var2,6297.00
2,01-Jan-2006 00:00:00,Var3,1425.00
3,01-Jan-2006 00:00:00,Var4,9829.00
4,01-Jan-2006 00:00:00,Var5,4293.00
...,...,...,...
3988315,31-Dec-2018 23:00:00,Var31,5290.00
3988316,31-Dec-2018 23:00:00,Var32,15044.93
3988317,31-Dec-2018 23:00:00,Var33,1169.00
3988318,31-Dec-2018 23:00:00,Var34,2834.56


In [9]:
rm data/electricity/datasets/mhlv.zip

In [10]:
mhlv_path = 'data/electricity/datasets/mhlv/M'

pathlib.Path(mhlv_path).mkdir(parents=True, exist_ok=True)
mhlv.to_csv(os.path.join(mhlv_path, 'df_y.csv'), index=False)

!zip -r data/electricity/datasets/mhlv.zip data/electricity/datasets/mhlv # -x "data/electricity/datasets/mhlv/raw/*"

  adding: data/electricity/datasets/mhlv/ (stored 0%)
  adding: data/electricity/datasets/mhlv/M/ (stored 0%)
  adding: data/electricity/datasets/mhlv/M/.ipynb_checkpoints/ (stored 0%)
  adding: data/electricity/datasets/mhlv/M/.ipynb_checkpoints/df_y-checkpoint.csv (deflated 84%)
  adding: data/electricity/datasets/mhlv/M/df_y-Copy1.csv (deflated 84%)
  adding: data/electricity/datasets/mhlv/M/df_y.csv (deflated 84%)
  adding: data/electricity/datasets/mhlv/raw/ (stored 0%)
  adding: data/electricity/datasets/mhlv/raw/MHLV.csv (deflated 63%)
  adding: data/electricity/datasets/mhlv/raw/MHL_test2.csv (deflated 57%)


In [11]:
# import subprocess

# proc = subprocess.run(["gsutil", "-m", "rsync", "data/electricity/datasets/", "gs://electricity-datasets"],
#                       capture_output=True)
# print(proc.stderr.decode())

In [12]:
# !rm data/electricity/datasets/mhlv.zip
# !rm -rf data/electricity/datasets/mhlv/M
# !gsutil -m rsync -r gs://electricity-datasets data/electricity/datasets/

In [13]:
ElectricityUnivariateDataModule.prepare_data()

INFO:dataset:Downloading datasets
INFO:dataset:Building synchronization state...
Starting synchronization...
Copying gs://electricity-datasets/mhlv.zip...
| [1/1 files][ 31.0 MiB/ 31.0 MiB] 100% Done   3.2 MiB/s ETA 00:00:00           
Operation completed over 1 objects/31.0 MiB.                                     

INFO:dataset:Unzipping datasets
INFO:dataset:Archive:  data/electricity/datasets/mhlv.zip



# Test the download

In [14]:
path = os.path.join('./data/electricity/datasets', 'mhlv', 'M/df_y.csv')
df = pd.read_csv(path)
df[-35:]

,ds,unique_id,y
3988285,31-Dec-2018 23:00:00,Var1,982.400000
3988286,31-Dec-2018 23:00:00,Var2,6707.000000
3988287,31-Dec-2018 23:00:00,Var3,1339.000000
3988288,31-Dec-2018 23:00:00,Var4,8973.000000
3988289,31-Dec-2018 23:00:00,Var5,3739.720000
3988290,31-Dec-2018 23:00:00,Var6,6684.000000
3988291,31-Dec-2018 23:00:00,Var7,5766.000000
3988292,31-Dec-2018 23:00:00,Var8,44231.183673
3988293,31-Dec-2018 23:00:00,Var9,3608.770000
3988294,31-Dec-2018 23:00:00,Var10,NaN


In [15]:
mhlv_test_ds = ElectricityUnivariateDataset(name='mhlv', split='train', split_start='2018-01-01', split_end='2019-01-01')


/workspace/any-quantile/dataset/datasets.py:105: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  self.df.fillna(method=self.fillna, inplace=True)


In [16]:
mhlv_test_ds.df[mhlv_test_ds.history_length:]

unique_id,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var26,Var27,Var28,Var29,Var30,Var31,Var32,Var33,Var34,Var35
ds,,,,,,,,,,,,,,,,,,,,,
2018-01-01 00:00:00,746.450012,6414.0,1249.0,8379.0,3424.489990,7311.0,5530.0,44142.347656,3146.500000,799.599976,...,11255.0,16763.589844,13654.082031,4500.0,5332.0,5142.0,15714.000000,1103.0,2734.270020,24635.320312
2018-01-01 01:00:00,626.280029,6153.0,1189.0,8010.0,3281.060059,7484.0,5496.0,42825.429688,3024.699951,783.200012,...,10956.0,16509.320312,13045.917969,4254.0,5162.0,4908.0,15745.000000,1058.0,2658.820068,23872.119141
2018-01-01 02:00:00,548.539978,5907.0,1142.0,7713.0,3172.750000,7459.0,5337.0,42276.468750,2931.399902,776.799988,...,10647.0,16109.379883,12706.122070,4047.0,5071.0,4564.0,15508.000000,1022.0,2582.760010,23194.890625
2018-01-01 03:00:00,519.030029,5918.0,1100.0,7568.0,3122.350098,7260.0,5293.0,42205.550781,2878.899902,779.200012,...,10431.0,16152.190430,12546.938477,3905.0,5013.0,4297.0,15324.000000,1010.0,2567.719971,23071.960938
2018-01-01 04:00:00,524.559998,6033.0,1090.0,7579.0,3111.679932,7285.0,5284.0,41956.898438,2896.899902,780.500000,...,10431.0,16205.080078,12437.754883,3815.0,4987.0,4134.0,15516.000000,1023.0,2562.350098,23267.900391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 19:00:00,1260.640015,7510.0,1655.0,9051.0,4191.430176,6612.0,6369.0,51404.152344,4011.399902,1030.849976,...,13117.0,16899.220703,16233.673828,6011.0,6502.0,5761.0,16766.439453,1435.0,3093.110107,30574.800781
2018-12-31 20:00:00,1116.699951,7134.0,1537.0,8825.0,4076.780029,6396.0,6215.0,49750.296875,3900.600098,1030.849976,...,12369.0,16559.330078,15440.816406,5505.0,6220.0,5379.0,16110.110352,1345.0,2990.830078,29479.669922
2018-12-31 21:00:00,1033.339966,7250.0,1471.0,9109.0,3999.010010,6567.0,6093.0,49220.000000,3720.399902,1030.849976,...,11937.0,16087.570312,15038.775391,5275.0,6023.0,5209.0,15577.030273,1286.0,2936.709961,28700.140625


In [17]:
ls data/electricity/datasets/

mhlv/  mhlv.zip


In [18]:
# !zip -r data/electricity/datasets/mhlv.zip data/electricity/datasets/mhlv

In [19]:
# mv data/electricity/datasets/mhlv data/electricity/datasets/mhlv-old

In [20]:
# !unzip data/electricity/datasets/mhlv.zip

In [21]:
!rm data/electricity/datasets/mhlv.zip
!rm -r data/electricity/datasets/mhlv
# !ls data/electricity/datasets/

In [22]:
from glob import glob

print("Downloading datasets")
datasets_path = "data/electricity/datasets/"
proc = subprocess.run(["gsutil", "-m", "rsync", "gs://electricity-datasets", datasets_path],
                      capture_output=True)
print(proc.stderr.decode())
datasets = glob(os.path.join(datasets_path, "*.zip"))

print("Unzipping datasets")
for d in datasets:
    proc = subprocess.run(["unzip", "-u", d], capture_output=True)
    print(proc.stdout.decode())

NameError: name 'subprocess' is not defined

In [ ]:
mhlv = ElectricityUnivariateDataset(name='mhlv', split='train', split_start=0, split_end=0.8, step=10)
self = mhlv

In [ ]:
self.num_windows = (~self.df.isnull()).sum(axis=0) - self.tot_window_len + 1

self.df.drop(self.df.columns[self.num_windows <= 0], axis=1, inplace=True)
self.num_windows = (~self.df.isnull()).sum(axis=0) - self.tot_window_len + 1
self.cum_num_windows = self.num_windows.cumsum()
self.num_nulls = self.df.isnull().sum(axis=0)


In [ ]:
index=54658-1

index = 259139 - 5467

column_idx = bisect.bisect_right(self.cum_num_windows, index)
window_idx = self.num_nulls[column_idx] + self.step * index - (self.step * self.cum_num_windows[column_idx-1] if column_idx > 0 else 0)

print('column_idx', column_idx)
print('window_idx', window_idx)

In [ ]:
self.df.values[window_idx:window_idx+self.tot_window_len, column_idx].astype(int)[:20]

In [ ]:
self.df.values[window_idx:window_idx+self.tot_window_len, column_idx].astype(int)[:20]

In [ ]:
len(self.df.values[89720:, 0])

In [ ]:
len(self.df)

In [ ]:
mhlv.cum_num_windows

In [ ]:
self.df.iloc[:,column_idx][:500]